In [2]:
!pip install xgboost

  Using cached https://files.pythonhosted.org/packages/4f/4c/4969b10939c4557ae46e5569d07c0c7ce772b3d6b9c1401a6ed07059fdee/xgboost-0.81.tar.gz
    Complete output from command python setup.py egg_info:
    ++ pwd
    + oldpath=/private/var/folders/y1/1sd072rd2h159qz2xrp00mnh0000gp/T/pip-build-84gaeect/xgboost
    + cd ./xgboost/
    + echo darwin17
    + grep -q darwin
    + LIB_XGBOOST=libxgboost.dylib
    + which g++-5
    + which g++-7
    + which clang++
    /usr/bin/clang++
    + export CC=clang
    + CC=clang
    + export CXX=clang++
    + CXX=clang++
    + make clean
    Makefile:31: MAKE [/Applications/Xcode.app/Contents/Developer/usr/bin/make] - checked OK
    rm -f -rf build build_plugin lib bin *~ */*~ */*/*~ */*/*/*~ */*.o */*/*.o */*/*/*.o #xgboost
    rm -f -rf build_tests *.gcov tests/cpp/xgboost_test
    if [ -d "R-package/src" ]; then \
    		cd R-package/src; \
    		rm -f -rf rabit src include dmlc-core amalgamation *.so *.dll; \
    		cd /private/var/folders/y1/1sd07

Command "python setup.py egg_info" failed with error code 1 in /private/var/folders/y1/1sd072rd2h159qz2xrp00mnh0000gp/T/pip-build-84gaeect/xgboost/
You are using pip version 9.0.1, however version 19.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install pystan

You are using pip version 9.0.1, however version 19.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
!pip install fbprophet

  Using cached https://files.pythonhosted.org/packages/9b/a1/fef4ce00acbc28e75c0d33f60c9777527c4295656903b00ac4c9525cef7f/fbprophet-0.4.post2.tar.gz
  Running setup.py bdist_wheel for fbprophet ... error
  Complete output from command /anaconda3/bin/python -u -c "import setuptools, tokenize;__file__='/private/var/folders/y1/1sd072rd2h159qz2xrp00mnh0000gp/T/pip-build-slbfgbz8/fbprophet/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /var/folders/y1/1sd072rd2h159qz2xrp00mnh0000gp/T/tmpkn7abk3spip-wheel- --python-tag cp36:
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib
  creating build/lib/fbprophet
  creating build/lib/fbprophet/stan_model
  DIAGNOSTIC(S) FROM PARSER:
  
  INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_861b75c6337e237650a61ae58c4385ef NOW.
  error: command 'gcc' failed with exit status 1
  
  ------------------

In [2]:
import warnings
warnings.filterwarnings("ignore")

# loading packages
# basic + dates 
import numpy as np
import pandas as pd
from pandas import datetime

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns # advanced vizs
%matplotlib inline

# statistics
from statsmodels.distributions.empirical_distribution import ECDF

# time series analysis
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# prophet by Facebook
from fbprophet import Prophet

In [3]:
testanswer=pd.read_csv("newtest.csv",  
                    low_memory = False)
A=testanswer.groupby('Store')['Sales'].aggregate(lambda x: list(x))

In [4]:
def rmspe(y, yhat):
    return np.sqrt(np.mean((yhat/y-1) ** 2))

In [5]:
df = pd.read_csv("newtrain.csv",  
                    low_memory = False)
df2 =  pd.read_csv("train.csv",  
                    low_memory = False)
df3 =  pd.read_csv("newtest.csv",  
                    low_memory = False)
# remove closed stores and those with no sales
df = df[(df["Open"] != 0) & (df['Sales'] != 0)]
i=1
rmspe1=[0]

for i in range(1,4):
    sales = df[df.Store == i].loc[:, ['Date', 'Sales','Promo']]

    sales['Date'] = pd.DatetimeIndex(sales['Date'])
    sales = sales.rename(columns = {'Date': 'ds',
                                'Sales': 'y'})
    state_dates = df2[(df2.Store==i) & ((df2.StateHoliday == 'a') | (df2.StateHoliday == 'b') & (df2.StateHoliday == 'c'))].loc[:, 'Date'].values
    school_dates = df2[(df2.Store==i) & (df2.SchoolHoliday == 1)].loc[:, 'Date'].values


    state = pd.DataFrame({'holiday': 'state_holiday',
                      'ds': pd.to_datetime(state_dates)})
    school = pd.DataFrame({'holiday': 'school_holiday',
                      'ds': pd.to_datetime(school_dates)})

    holidays = pd.concat((state, school)) 

    my_model = Prophet(holidays = holidays)
    my_model.add_regressor('Promo')
    my_model.fit(sales)

    future_dates = my_model.make_future_dataframe(periods = 22*7)
    a=np.array(pd.concat((df[df.Store == i].loc[:, 'Promo'], df3[df3.Store == i].loc[:, 'Promo'])))
    b=np.array((0,0,0))
    c=np.append(a,b)
    future_dates['Promo']=c
    forecast = my_model.predict(future_dates)
    fc = forecast[['ds', 'yhat']].rename(columns = {'Date': 'ds', 'Forecast': 'yhat'})
    yhat=fc['yhat'][606:757]
    y=A[i]
    yhat=np.array(yhat)
    y=np.array(y)
    y=y[np.where(y>0)]
    yhat=yhat[np.where(y>0)]
    rmspe1.append(rmspe(y,yhat))
    print(rmspe1)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


[0, 0.4664448846836518]


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


[0, 0.4664448846836518, 0.5380062469283732]


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


[0, 0.4664448846836518, 0.5380062469283732, 0.4822675808402173]


In [6]:
df = pd.read_csv("C://Users//Owner//Desktop//newtrain.csv",  
                    low_memory = False)
df2 =  pd.read_csv("C://Users//Owner//Desktop//train.csv",  
                    low_memory = False)
df3 =  pd.read_csv("C://Users//Owner//Desktop//newtest.csv",  
                    low_memory = False)
# remove closed stores and those with no sales
df = df[(df["Open"] != 0) & (df['Sales'] != 0)]
i=1
rmspe1=[0]

for i in range(1,4):
    sales = df[df.Store == i].loc[:, ['Date', 'Sales']]

    sales['Date'] = pd.DatetimeIndex(sales['Date'])
    sales = sales.rename(columns = {'Date': 'ds',
                                'Sales': 'y'})
    state_dates = df2[(df2.Store==i) & ((df2.StateHoliday == 'a') | (df2.StateHoliday == 'b') & (df2.StateHoliday == 'c'))].loc[:, 'Date'].values
    school_dates = df2[(df2.Store==i) & (df2.SchoolHoliday == 1)].loc[:, 'Date'].values


    state = pd.DataFrame({'holiday': 'state_holiday',
                      'ds': pd.to_datetime(state_dates)})
    school = pd.DataFrame({'holiday': 'school_holiday',
                      'ds': pd.to_datetime(school_dates)})

    holidays = pd.concat((state, school)) 

    my_model = Prophet(holidays = holidays)
   
    my_model.fit(sales)

    future_dates = my_model.make_future_dataframe(periods = 22*7)
   
  
    forecast = my_model.predict(future_dates)
    fc = forecast[['ds', 'yhat']].rename(columns = {'Date': 'ds', 'Forecast': 'yhat'})
    yhat=fc['yhat'][606:757]
    y=A[i]
    yhat=np.array(yhat)
    y=np.array(y)
    y=y[np.where(y>0)]
    yhat=yhat[np.where(y>0)]
    rmspe1.append(rmspe(y,yhat))
    print(rmspe1)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


[0, 0.37956624798575783]


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


[0, 0.37956624798575783, 0.45942839745182357]


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


[0, 0.37956624798575783, 0.45942839745182357, 0.39852793758581084]
